In [1]:
# Loading Packages

# EDA
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
from numpy import sort
from collections import Counter

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_curve, auc, log_loss, roc_auc_score 
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
import xgboost

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

# Settings
import pickle
import warnings
warnings.filterwarnings("ignore")

In [2]:
ds = pd.read_csv('train.csv')
ds = ds.drop_duplicates()
X = ds.drop('label', axis=1)
y = ds.label.values
x_train_val, x_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_val,y_train_val, test_size=0.10,random_state=42)
x_train = x_train/255
x_val = x_val/255
x_test= x_test/255
#model

def model_score(model, x_train, y_train, x_val,y_val,x_test,y_test):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'AUC', 'Recall', 'Precision']
    )

    callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
                EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=12, restore_best_weights=True)]

    
    print("\n MODEL BUILDING ============================================================================ \n")

    history = model.fit(
        x_train, y_train, 
        epochs=128, 
        batch_size = 32, 
        validation_data=(x_val,y_val),
        callbacks=callbacks
    )

    print("\n MODEL EVALUATION ON TEST SET ============================================================================ \n")

    score = model.evaluate(x_test, y_test, batch_size=32)

    return [model, history, score]


def pred_test_to_csv(model, name):
    test_ds = pd.read_csv("test.csv")
    Id = test_ds['Id'][:]
    test_ds = test_ds.drop("Id", axis=1)
    test_ds = test_ds/255
    pred = model.predict(test_ds)
    pred_df = pd.DataFrame({'Id':Id.values, 'Predicted':pred.flatten()}, columns=['Id', 'Predicted'])
    print(pred_df)
    pred_df.to_csv(f'{name}.csv', index=False)
    return True


In [ ]:
records = {}

In [3]:
model = tf.keras.Sequential([
    layers.Reshape(target_shape=(20, 20, 1), input_shape=(400,)),
    layers.Conv2D(32, (3, 3), activation = 'relu', kernel_regularizer=l2(0.0000001), padding='same'),
    layers.BatchNormalization(),
    layers.SpatialDropout2D(0.4),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    layers.Conv2D(64, (3, 3), activation = 'relu', kernel_regularizer=l2(0.0000001), padding='same'),
    layers.BatchNormalization(),
    layers.SpatialDropout2D(0.4),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    layers.Conv2D(128, (3, 3), activation = 'relu', kernel_regularizer=l2(0.0000001), padding='same'),
    layers.BatchNormalization(),
    layers.SpatialDropout2D(0.4),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    layers.Conv2D(256, (3, 3), activation = 'relu', kernel_regularizer=l2(0.0000001), padding='same'),
    layers.BatchNormalization(),
    layers.SpatialDropout2D(0.4),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    layers.Conv2D(256, (3, 3), activation = 'relu', kernel_regularizer=l2(0.0000001), padding='same'),
    layers.BatchNormalization(),
    #layers.SpatialDropout2D(0.5),
    #layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),
    layers.Flatten(),
    layers.Dropout(0.6),
    #layers.Dropout(0.6, seed=RANDOM_STATE),
    layers.Dense(128, activation='relu', kernel_regularizer=l2(0.0000001)),
    layers.Dropout(0.6),
    #layers.Dense(128, activation='relu', kernel_regularizer=l2(0.000001)),
    layers.Dense(16, activation='relu', kernel_regularizer=l2(0.0000001)),
    layers.Dense(1, activation='sigmoid')
])

model_CNN = model_score(model, x_train, y_train, x_val, y_val, x_test,y_test)


 MODEL BUILDING ============================================================================ 

Epoch 1/128
1620/1620 [==============================] - 44s 20ms/step - loss: 0.2793 - accuracy: 0.9140 - auc: 0.7129 - recall: 0.0854 - precision: 0.6584 - val_loss: 0.1811 - val_accuracy: 0.9338 - val_auc: 0.9135 - val_recall: 0.3205 - val_precision: 0.8608
Epoch 2/128
1620/1620 [==============================] - 32s 20ms/step - loss: 0.1982 - accuracy: 0.9332 - auc: 0.8759 - recall: 0.3659 - precision: 0.7671 - val_loss: 0.1378 - val_accuracy: 0.9481 - val_auc: 0.9502 - val_recall: 0.4914 - val_precision: 0.8828
Epoch 3/128
1620/1620 [==============================] - 35s 22ms/step - loss: 0.1639 - accuracy: 0.9453 - auc: 0.9183 - recall: 0.5185 - precision: 0.8017 - val_loss: 0.1190 - val_accuracy: 0.9578 - val_auc: 0.9612 - val_recall: 0.5681 - val_precision: 0.9427
Epoch 4/128
1620/1620 [==============================] - 37s 23ms/step - loss: 0.1445 - accuracy: 0.9513 - auc: 0.9359 - 